In [2]:
# importing all libraries 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 

from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings 
import re

In [3]:
# Load the datasets
a1 = pd.read_csv('case_study1.csv')
a2 = pd.read_csv('case_study2.csv')

In [5]:
df1 = a1.copy()
df2 = a2.copy()

In [6]:
print(df1.shape)
print(df2.shape)

(51336, 26)
(51336, 62)


In [7]:
# Remove null values
f1 = df1.loc[df1['Age_Oldest_TL'] != -99999]

In [9]:
columns_to_be_removed = []

for i in df2.columns:
    if df2.loc[df2[i] == -9999].shape[0] > 10000:
        columns_to_be_removed.append(i)

In [11]:
df2 = df2.drop(columns_to_be_removed, axis = 1)

In [13]:
for i in df2.columns:
    df2 = df2.loc[ df2[i] != -9999]
    

In [15]:
for i in list(df1.columns):
    if i in list(df2.columns):
        print (i)

PROSPECTID


In [16]:
# Merge the two dataframes, inner join so that no nulls are present 
df = pd.merge(df1, df2, how = 'inner', left_on = ['PROSPECTID'], right_on = ['PROSPECTID'])

In [17]:
df.shape

(51336, 87)

In [19]:
# check how many columns are categorical 
for i in df.columns:
    if df[i].dtype == 'object':
        print(i)

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [22]:
# Chi square test 
for i in ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2','first_prod_enq2']:
    chi2, pval, _, _ = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
    print(i, '---', pval)

MARITALSTATUS --- 2.7588304433709322e-257
EDUCATION --- 8.464676085080213e-38
GENDER --- 0.0002450667989541709
last_prod_enq2 --- 0.0
first_prod_enq2 --- 0.0


In [23]:
# VIF for numerical columns 
numeric_columns = []
for i in df.columns:
    if df[i].dtype != 'object' and i not in ['PROSPECTID','Approved_Flag']:
        numeric_columns.append(i)

In [25]:
# VIF sequential check 
vif_data = df[numeric_columns]
total_columns = vif_data.shape[1]
columns_to_be_kept = []
column_index = 0

In [26]:
for i in range(0, total_columns):
    vif_value = variance_inflation_factor(vif_data, column_index)
    print(column_index, '---', vif_value)
    if vif_value <= 6:
        columns_to_be_kept.append(numeric_columns[i])
        column_index = column_index+1
    else:
        vif_data = vif_data.drop([numeric_columns[i]], axis=1)

/nfs/cms/mtech23/lute.gaurav/.local/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf


/nfs/cms/mtech23/lute.gaurav/.local/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf
0 --- 11.142024054348182
0 --- 8.335916685517912
0 --- 6.463010922496328
0 --- 5.534338081913364
1 --- 2.768696209570132


/nfs/cms/mtech23/lute.gaurav/.local/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 --- inf
2 --- 2462.4419800557143
2 --- 8.248565731413791
2 --- 3.7005074426983775
3 --- 5.345861316163595
4 --- 5.4874084932992755
5 --- 2.0070069640038106


/nfs/cms/mtech23/lute.gaurav/.local/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


6 --- inf
6 --- 4.8253969617799335
7 --- 21.35788522795101
7 --- 33.73120640745207
7 --- 4.458924841554711
8 --- 3.0256428589655346
9 --- 2.8232264699260554
10 --- 4.559577060503653
11 --- 2.186805944220353
12 --- 10304.74363772914
12 --- 1.016330610711997
13 --- 1.3077371916871499
14 --- 594916843.2536781
14 --- 6697913.387088732
14 --- 8.833627853741577
14 --- 5.831534786166029
15 --- 5.411420161253207


/nfs/cms/mtech23/lute.gaurav/.local/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


16 --- inf
16 --- 7.59738152314555
16 --- 1.5468167622420932
17 --- 8.337257020955212
17 --- 2.178453754790816
18 --- 8.831456557200411
18 --- 1.6316029579318194
19 --- 7.07138831138449
19 --- 15.543852815081975
19 --- 1.9680509535969277
20 --- 1.5652951537350022
21 --- 2.546399219282383
22 --- 3.103617519649693
23 --- 2.1918035493390193
24 --- 7.384677371724363
24 --- 2.074497017027807
25 --- 2.7237404160585936
26 --- 6.293500947046617
26 --- 2.709506792824964
27 --- 4.974334443038774
28 --- 345919259.0287266
28 --- 2992625835.601257
28 --- 11996291120.704754
28 --- 1431937079.2660658
28 --- 1787956513.2901745
28 --- 5369011857.07013
28 --- 500717135.084179
28 --- 8358.042814974533
28 --- 668774090.3879377
28 --- 786060777.1803054
28 --- 1.3762266586619558
29 --- 21.239202084829607
29 --- 2.870772683673689
30 --- 3.3473430734875014
31 --- 9.265637089357964
31 --- 6.496060757304039
31 --- 1.0114109560406566
32 --- 67.02279235811868
32 --- 3.1375569461047674
33 --- 34.109022299576566
33

In [29]:
# Check Anova for columns_to_be_kept
from scipy.stats import f_oneway

columns_to_be_kept_numerical = []
for i in columns_to_be_kept :
    a = list(df[i])
    b = list(df['Approved_Flag'])
    group_P1 = [value for value, group in zip(a, b) if group=='P1']
    group_P2 = [value for value, group in zip(a, b) if group=='P2']
    group_P3 = [value for value, group in zip(a, b) if group=='P3']
    group_P4 = [value for value, group in zip(a, b) if group=='P4']
    f_statistic, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)
    if p_value <= 0.05:
        columns_to_be_kept_numerical.append(i)
        

In [33]:
# Feature selection done for categorical and numerical features
# listing all final feature 
features = columns_to_be_kept_numerical + ['MARITALSTATUS', 'EDUCATION','GENDER', 'last_prod_enq2', 'first_prod_enq2']
df = df[features + ['Approved_Flag']]

In [34]:
# Label encoding for the categorical feature 
['MARITALSTATUS', 'EDUCATION', 'GENDER' , 'last_prod_enq2' ,'first_prod_enq2']

['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

In [36]:
df['MARITALSTATUS'].unique()
df['EDUCATION'].unique()
df['GENDER'].unique()
df['last_prod_enq2'].unique()
df['first_prod_enq2'].unique()

array(['PL', 'ConsumerLoan', 'others', 'AL', 'HL', 'CC'], dtype=object)

In [37]:
# Ordinal feature -- EDUCATION
# SSC: 1
# 12TH: 2
# GRADUATE: 3
# UNDER GRADUATE : 3
# POST-GRADUATE : 4
# OTHERS: 1
# PROFESSIONAL: 3

In [38]:
df['EDUCATION'].unique()

array(['12TH', 'GRADUATE', 'SSC', 'POST-GRADUATE', 'UNDER GRADUATE',
       'OTHERS', 'PROFESSIONAL'], dtype=object)

In [39]:
# Other has to be verified by the buissiness end user 

In [41]:
df.loc[df['EDUCATION']=='SSC',['EDUCATION']] == 1
df.loc[df['EDUCATION']=='SSC',['EDUCATION']] = 1
df.loc[df['EDUCATION']=='12TH',['EDUCATION']]=2
df.loc[df['EDUCATION']=='GRADUATE',['EDUCATION']]=3
df.loc[df['EDUCATION']=='UNDER GRADUATE',['EDUCATION']]=3
df.loc[df['EDUCATION']=='POST-GRADUATE',['EDUCATION']]=4
df.loc[df['EDUCATION']=='OTHERS',['EDUCATION']]=1
df.loc[df['EDUCATION']=='PROFESSIONAL',['EDUCATION']]=3

In [43]:
df['EDUCATION'].value_counts()
df['EDUCATION'] = df['EDUCATION'].astype(int)
df.info()

/tmp/ipykernel_348169/2065880593.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EDUCATION'] = df['EDUCATION'].astype(int)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 42 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            51336 non-null  float64
 1   pct_tl_closed_L6M          51336 non-null  float64
 2   Tot_TL_closed_L12M         51336 non-null  int64  
 3   pct_tl_open_L12M           51336 non-null  float64
 4   pct_tl_closed_L12M         51336 non-null  float64
 5   Tot_Missed_Pmnt            51336 non-null  int64  
 6   CC_TL                      51336 non-null  int64  
 7   Home_TL                    51336 non-null  int64  
 8   PL_TL                      51336 non-null  int64  
 9   Secured_TL                 51336 non-null  int64  
 10  Unsecured_TL               51336 non-null  int64  
 11  Other_TL                   51336 non-null  int64  
 12  Age_Newest_TL              51336 non-null  int64  
 13  time_since_recent_payment  51336 non-null  int

In [44]:
df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS','GENDER','last_prod_enq2' ,'first_prod_enq2'])

In [45]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 54 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               51336 non-null  float64
 1   pct_tl_closed_L6M             51336 non-null  float64
 2   Tot_TL_closed_L12M            51336 non-null  int64  
 3   pct_tl_open_L12M              51336 non-null  float64
 4   pct_tl_closed_L12M            51336 non-null  float64
 5   Tot_Missed_Pmnt               51336 non-null  int64  
 6   CC_TL                         51336 non-null  int64  
 7   Home_TL                       51336 non-null  int64  
 8   PL_TL                         51336 non-null  int64  
 9   Secured_TL                    51336 non-null  int64  
 10  Unsecured_TL                  51336 non-null  int64  
 11  Other_TL                      51336 non-null  int64  
 12  Age_Newest_TL                 51336 non-null  int64  
 13  t

In [46]:
k = df_encoded.describe()

In [47]:
k

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_open_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,...,NETMONTHLYINCOME,Time_With_Curr_Empr,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,EDUCATION
count,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,...,5.133600e+04,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000
mean,0.184574,0.089095,0.736851,0.395184,0.149989,0.546751,0.124981,0.070146,0.282511,2.844904,...,2.642419e+04,110.707846,0.089469,0.167874,0.170492,0.056302,-45127.943635,0.271116,0.052887,2.286836
std,0.297414,0.205635,1.454120,0.391930,0.257267,1.085529,0.505201,0.340861,0.858168,6.187177,...,2.002711e+04,76.046831,0.285423,0.373758,0.350209,0.213506,49795.784556,0.444540,0.223810,0.875756
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,-99999.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.800000e+04,61.000000,0.000000,0.000000,0.000000,0.000000,-99999.000000,0.000000,0.000000,2.000000
50%,0.000000,0.000000,0.000000,0.333000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,2.300000e+04,93.000000,0.000000,0.000000,0.000000,0.000000,0.333000,0.000000,0.000000,2.000000
75%,0.308000,0.053000,1.000000,0.750000,0.250000,1.000000,0.000000,0.000000,0.000000,3.000000,...,3.000000e+04,131.000000,0.000000,0.000000,0.000000,0.000000,2.164250,1.000000,0.000000,3.000000
max,1.000000,1.000000,39.000000,1.000000,1.000000,34.000000,27.000000,10.000000,29.000000,235.000000,...,2.500000e+06,1020.000000,1.000000,1.000000,1.000000,1.000000,173800.000000,1.000000,1.000000,4.000000


In [48]:
# MAchine learning model fitting 

In [49]:
# 1.Random Forest 
x = df_encoded.drop(['Approved_Flag'], axis=1)
y = df_encoded['Approved_Flag']

In [50]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [54]:
rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)

In [55]:
rf_classifier

RandomForestClassifier(n_estimators=200, random_state=42)

In [56]:
rf_classifier.fit(x_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [57]:
y_pred = rf_classifier.predict(x_test)

In [59]:
# Accuracy 
accuracy = accuracy_score(y_test, y_pred)
print()
print(f"Accuracy:{accuracy}")
print()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)


Accuracy:0.7412349045578497



In [60]:
for i, v in enumerate(['P1','P2','P3','P4']):
    print(f"Class {v}")
    print(f"Precision: {precision[i]}")
    print(f'Recall :{recall[i]}')
    print(f"F1 score:{f1_score[i]}")
    print()

Class P1
Precision: 0.780373831775701
Recall :0.44140969162995597
F1 score:0.5638716938660664

Class P2
Precision: 0.7644285896292521
Recall :0.9405862987929142
F1 score:0.843407365757661

Class P3
Precision: 0.46434494195688225
Recall :0.18229166666666666
F1 score:0.26180458158017766

Class P4
Precision: 0.706984667802385
Recall :0.6814449917898193
F1 score:0.6939799331103679



In [61]:
# 2 xgboost 
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [62]:
xgb_classifier = xgb.XGBClassifier(objective='multi: softmax', num_class=4)

In [65]:
x = df_encoded.drop(['Approved_Flag'], axis=1)
y = df_encoded['Approved_Flag']

In [66]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [72]:
x_train, x_test, y_train, y_test = train_test_split(x,y_encoded,test_size=0.3, random_state=42)

In [73]:
xgb_classifier.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=4,
              num_parallel_tree=None, ...)

In [74]:
y_pred = xgb_classifier.predict(x_test)

In [75]:
accuracy = accuracy_score(y_test, y_pred)

In [76]:
accuracy

0.7487825465878839

In [80]:
precision1, recall1, f1_score1, _ = precision_recall_fscore_support(y_test, y_pred)

In [81]:
for i, v in enumerate(['p1','p2','p3','p4']):
    print(f"Class{v}:")
    print(f"Precision: {precision1[i]}")
    print(f"Recall: {recall1[i]}")
    print(f"F1 Score: {f1_score1[i]}")
    print()

Classp1:
Precision: 0.731404958677686
Recall: 0.5169392523364486
F1 Score: 0.6057494866529775

Classp2:
Precision: 0.7884032114183764
Recall: 0.9170903808239078
F1 Score: 0.8478917829903583

Classp3:
Precision: 0.4480062548866302
Recall: 0.2515364354697103
F1 Score: 0.3221816137194265

Classp4:
Precision: 0.7262044653349001
Recall: 0.6967305524239008
F1 Score: 0.7111622554660529



In [82]:
# 3.Decision Tree
from sklearn.tree import DecisionTreeClassifier
y = df_encoded['Approved_Flag']
x = df_encoded.drop(['Approved_Flag'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=42)

In [83]:
dt_model = DecisionTreeClassifier(max_depth=20, min_samples_split=10)
dt_model.fit(x_train, y_train)
y_pred = dt_model.predict(x_test)

In [84]:
accuracy = accuracy_score(y_test, y_pred)
print()
print(f"Accuracy:{accuracy:.2f}")
print()


Accuracy:0.68



In [86]:
precission2, recall2, f1_score2, _ = precision_recall_fscore_support(y_test,y_pred)

In [87]:
for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}")
    print(f"Precision2:{precision[i]}")
    print(f'Recall2: {recall[i]}')
    print(f"F1 Score2: {f1_score[i]}")

Class p1
Precision2:0.731404958677686
Recall2: 0.5136563876651983
F1 Score2: 0.5186832740213523
Class p2
Precision2:0.7884032114183764
Recall2: 0.8112556827088886
F1 Score2: 0.7966440886699507
Class p3
Precision2:0.4480062548866302
Recall2: 0.3177083333333333
F1 Score2: 0.3298411625549172
Class p4
Precision2:0.7262044653349001
Recall2: 0.5944170771756979
F1 Score2: 0.6195977749251177
